<a href="https://colab.research.google.com/github/Hyung-Z/music-recommeder/blob/main/%EA%B0%90%EC%84%B1_%EB%B6%84%EC%84%9D_%EC%B1%97%EB%B4%87_%EC%8B%A4%EC%A0%9C_%EB%8D%B0%EC%9D%B4%ED%84%B0_%ED%95%99%EC%8A%B5_%ED%8C%8C%EC%9D%B4%ED%94%84%EB%9D%BC%EC%9D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# =================================================================================
# 감성 분석 기반 음악 추천 챗봇 - 최종 통합 파이프라인 (v1.3)
# =================================================================================
# 이 코드는 아래의 개선 사항을 모두 포함한 최종 버전입니다.
# 1. 사용자가 직접 업로드한 JSON 파일(훈련/검증)을 사용하도록 수정
# 2. 과대적합 방지 (조기 종료) 및 wandb 로깅 비활성화
# 3. 감정/문맥 듀얼 모델을 사용한 하이브리드 추천 시스템
# 4. 생성 파일 버전 관리 기능 추가 (v1.2)
#
# ※ Google Colab에서 'GPU 런타임'으로 실행해야 합니다.
# =================================================================================

# ## 0단계: 필수 라이브러리 설치 및 버전 정의
# ---------------------------------------------------------------------------------
!pip install transformers[torch] datasets scikit-learn pandas openpyxl requests sentence-transformers -q

import json
import pandas as pd
import numpy as np
import torch
from sklearn.metrics import accuracy_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sentence_transformers import SentenceTransformer
from datasets import Dataset
import os
import requests
from glob import glob

# ★★★★★ 버전 정의 ★★★★★
VERSION = "v1.5"

# =================================================================================
# ## 1단계: 실제 데이터 준비 및 전처리 (훈련/검증 데이터 분리)
# =================================================================================
print("="*50)
print("## 1단계: 데이터 준비 및 전처리 시작")
print("="*50)


# ★★★★★ 수정된 부분 ★★★★★
# 1-1. 사용자가 직접 업로드한 JSON 파일 경로 설정
# ※ 실행 전, Colab에 'train.json'과 'validation.json' 파일을 직접 업로드해주세요.
train_file_path = '/content/drive/MyDrive/chatbot_project/train.json'
valid_file_path = '/content/drive/MyDrive/chatbot_project/validation.json'

# 1-2. 훈련 및 검증 JSON 파일 목록 가져오기
train_files = [train_file_path] if os.path.exists(train_file_path) else []
valid_files = [valid_file_path] if os.path.exists(valid_file_path) else []

if not train_files:
    print(f"경고: 훈련 데이터 파일 '{train_file_path}'가 없습니다.")
if not valid_files:
    print(f"경고: 검증 데이터 파일 '{valid_file_path}'가 없습니다.")

# 1-3. 수정된 감정 라벨 매핑 정의
emotion_map = {
    '분노': ['E10', 'E11', 'E12', 'E13', 'E14', 'E15', 'E16', 'E17', 'E18', 'E19'],
    '슬픔': ['E20', 'E21', 'E22', 'E23', 'E24', 'E25', 'E26', 'E27', 'E28', 'E29'],
    '불안': ['E30', 'E31', 'E32', 'E33', 'E34', 'E35', 'E36', 'E37', 'E38', 'E39'],
    '상처': ['E40', 'E41', 'E42', 'E43', 'E44', 'E45', 'E46', 'E47', 'E48', 'E49'],
    '당황': ['E50', 'E51', 'E52', 'E53', 'E54', 'E55', 'E56', 'E57', 'E58', 'E59'],
    '기쁨': ['E60', 'E61', 'E62', 'E63', 'E64', 'E65', 'E66', 'E67', 'E68', 'E69'],
}
code_to_emotion = {code: emotion for emotion, codes in emotion_map.items() for code in codes}
labels = list(emotion_map.keys())
label_to_id = {label: i for i, label in enumerate(labels)}
id_to_label = {i: label for i, label in enumerate(labels)}

# 1-4. 데이터프레임 생성 함수
def create_dataframe_from_files(file_paths):
    processed_data = []
    # 파일이 단 하나일 수 있으므로, 파일 리스트를 순회합니다.
    for file_path in file_paths:
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                # 파일 내용이 JSON 객체들의 리스트라고 가정합니다.
                data_list = json.load(f)
            for data in data_list:
                emotion_code = data['profile']['emotion']['type']
                main_emotion = code_to_emotion.get(emotion_code)
                text = data['talk']['content']['HS01']
                if main_emotion and text:
                    processed_data.append({'text': text, 'label_name': main_emotion, 'label': label_to_id[main_emotion]})
        except Exception:
            continue
    return pd.DataFrame(processed_data)

# 1-5. 훈련 및 검증 데이터프레임 생성
train_df = create_dataframe_from_files(train_files)
eval_df = create_dataframe_from_files(valid_files)

if train_df.empty or eval_df.empty:
    print("\n오류: 훈련 또는 검증 데이터를 찾지 못했습니다. 프로그램 실행을 중단합니다.")
    print("해결 방법: Colab에 'train.json'과 'validation.json' 파일을 올렸는지 확인하세요.")
else:
    print(f"총 {len(train_df)}개의 훈련 데이터와 {len(eval_df)}개의 검증 데이터를 로드했습니다.")
    print("1단계 완료.\n")


# =================================================================================
# ## 2단계: 감정 분류 모델 실제 파인튜닝 (업그레이드)
# =================================================================================
if not train_df.empty and not eval_df.empty:
    print("="*50)
    print("## 2단계: 감정 분류 모델 파인튜닝 시작")
    print("="*50)

    MODEL_NAME = 'beomi/KcELECTRA-base-v2022'
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    train_dataset = Dataset.from_pandas(train_df).map(lambda e: tokenizer(e['text'], padding='max_length', truncation=True, max_length=128), batched=True)
    eval_dataset = Dataset.from_pandas(eval_df).map(lambda e: tokenizer(e['text'], padding='max_length', truncation=True, max_length=128), batched=True)

    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(labels), id2label=id_to_label, label2id=label_to_id)

    training_args = TrainingArguments(
        output_dir=f'/content/drive/MyDrive/chatbot_project/results_{VERSION}',
        num_train_epochs=5,
        learning_rate=3e-5,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        logging_dir=f'/content/drive/MyDrive/chatbot_project/logs_{VERSION}',
        logging_steps=500,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        save_total_limit=1,
        report_to="none",
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=lambda p: {'accuracy': accuracy_score(p.label_ids, np.argmax(p.predictions, axis=1))},
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )
    print("모델 파인튜닝을 시작합니다...")
    trainer.train()
    model_save_path = f'/content/drive/MyDrive/chatbot_project/my_emotion_model_{VERSION}'
    trainer.save_model(model_save_path)
    tokenizer.save_pretrained(model_save_path)
    print(f"\n2단계 완료: 파인튜닝된 모델이 '{model_save_path}' 폴더에 저장되었습니다.\n")


# =================================================================================
# ## 3단계: 노래 가사 듀얼 임베딩 데이터베이스 구축
# =================================================================================
if os.path.exists(f'/content/drive/MyDrive/chatbot_project/my_emotion_model_{VERSION}'):
    print("="*50)
    print("## 3단계: 노래 가사 듀얼 임베딩 데이터베이스 구축 시작")
    print("="*50)

    emotion_model_path = f'/content/drive/MyDrive/chatbot_project/my_emotion_model_{VERSION}'
    emotion_tokenizer = AutoTokenizer.from_pretrained(emotion_model_path)
    emotion_model = AutoModelForSequenceClassification.from_pretrained(emotion_model_path, output_hidden_states=True)
    emotion_model.eval()

    context_model = SentenceTransformer('BM-K/KoSimCSE-roberta')
    print("감정 분석 모델과 문맥 분석 모델을 로드했습니다.")

    lyrics_data_url = "https://raw.githubusercontent.com/Hyung-Z/tvshowgame/refs/heads/main/data.json"
    try:
        response = requests.get(lyrics_data_url)
        response.raise_for_status()
        user_lyrics_data = response.json()
        print(f"URL에서 {len(user_lyrics_data)}개의 노래 데이터를 성공적으로 불러왔습니다.")
    except Exception as e:
        print(f"URL에서 데이터를 불러오는 데 실패했습니다: {e}")
        user_lyrics_data = {}

    song_database = []
    all_lyrics = [
        song_data[3] for song_data in user_lyrics_data.values()
        if isinstance(song_data, list) and len(song_data) > 3 and isinstance(song_data[3], str)
    ]
    lyrics_context_embeddings = context_model.encode(all_lyrics, batch_size=32, show_progress_bar=True)

    idx = 0
    lyrics_emotion_embeddings = []
    for song_data in user_lyrics_data.values():
        try:
            if not (isinstance(song_data, list) and len(song_data) > 3 and isinstance(song_data[3], str)): continue

            title = song_data[1][0] if song_data[1] else "제목 없음"
            artist = song_data[2]
            lyrics = song_data[3]
            videoId = song_data[0]

            inputs = emotion_tokenizer(lyrics, return_tensors="pt", truncation=True, padding=True, max_length=128)
            with torch.no_grad(): outputs = emotion_model(**inputs)

            prediction = torch.argmax(outputs.logits, dim=-1).item()
            emotion_label = emotion_model.config.id2label[prediction]
            emotion_embedding = outputs.hidden_states[-1][:, 0, :].numpy()

            context_embedding = lyrics_context_embeddings[idx]

            song_info = {"artist": artist, "title": title, "lyrics": lyrics, "emotion": emotion_label, "videoId": videoId}
            song_database.append(song_info)
            lyrics_emotion_embeddings.append(emotion_embedding)
            idx += 1
        except Exception: continue

    if song_database:
        lyrics_emotion_embeddings = np.vstack(lyrics_emotion_embeddings)
        np.save(f'/content/drive/MyDrive/chatbot_project/lyrics_emotion_embeddings_{VERSION}.npy', lyrics_emotion_embeddings)
        np.save(f'/content/drive/MyDrive/chatbot_project/lyrics_context_embeddings_{VERSION}.npy', lyrics_context_embeddings)
        with open(f'/content/drive/MyDrive/chatbot_project/song_database_{VERSION}.json', 'w', encoding='utf-8') as f:
            json.dump(song_database, f, ensure_ascii=False, indent=4)
        print(f"\n총 {len(song_database)}곡의 가사로 듀얼 임베딩 데이터베이스를 구축했습니다.")
        print("3단계 완료.\n")
    else:
        print("\n오류: 노래 가사 데이터베이스를 구축하지 못했습니다.")


# =================================================================================
# ## 4단계: 업그레이드된 감성 분석 챗봇 실행
# =================================================================================
if os.path.exists(f'/content/drive/MyDrive/chatbot_project/song_database_{VERSION}.json'):
    print("="*50)
    print(f"## 4단계: 업그레이드된 감성 분석 챗봇 실행 시작 (Version: {VERSION})")
    print("="*50)

    db_emotion_vectors = np.load(f'/content/drive/MyDrive/chatbot_project/lyrics_emotion_embeddings_{VERSION}.npy')
    db_context_vectors = np.load(f'/content/drive/MyDrive/chatbot_project/lyrics_context_embeddings_{VERSION}.npy')
    with open(f'/content/drive/MyDrive/chatbot_project/song_database_{VERSION}.json', 'r', encoding='utf-8') as f:
        db_song_info = json.load(f)

    def cosine_similarity(vec1, vec2_matrix):
        vec1_norm = np.linalg.norm(vec1)
        vec2_matrix_norm = np.linalg.norm(vec2_matrix, axis=1)
        epsilon = 1e-8
        return np.dot(vec2_matrix, vec1) / (vec1_norm * vec2_matrix_norm + epsilon)

    def recommend_music(user_input, top_k=3, emotion_weight=0.3, context_weight=0.7):
        print(f"\n사용자 입력: '{user_input}'")

        inputs = emotion_tokenizer(user_input, return_tensors="pt", truncation=True, padding=True, max_length=128)
        with torch.no_grad(): outputs = emotion_model(**inputs)

        user_emotion_label = emotion_model.config.id2label[torch.argmax(outputs.logits, dim=-1).item()]
        user_emotion_vector = outputs.hidden_states[-1][:, 0, :].numpy().flatten()
        user_context_vector = context_model.encode(user_input)

        print(f"-> 입력 감정 분석 결과: '{user_emotion_label}'")

        candidate_indices = [i for i, song in enumerate(db_song_info) if song['emotion'] == user_emotion_label]

        if not candidate_indices:
            print("-> 아쉽지만, 현재 감정과 일치하는 노래를 찾지 못했어요.")
            return

        cand_emotion_vectors = db_emotion_vectors[candidate_indices]
        cand_context_vectors = db_context_vectors[candidate_indices]
        candidate_songs = [db_song_info[i] for i in candidate_indices]

        emotion_sims = cosine_similarity(user_emotion_vector, cand_emotion_vectors)
        context_sims = cosine_similarity(user_context_vector, cand_context_vectors)

        total_scores = (emotion_weight * emotion_sims) + (context_weight * context_sims)
        sorted_indices = np.argsort(total_scores)[::-1]

        print("\n--- 최종 추천 결과 ---")
        for i in range(min(top_k, len(sorted_indices))):
            song_idx_in_cand = sorted_indices[i]
            song = candidate_songs[song_idx_in_cand]
            final_score = total_scores[song_idx_in_cand]
            print(f"{i+1}. {song['artist']} - {song['title']} (종합 점수: {final_score:.4f})")
            print(f"   ㄴ 가사: \"{song['lyrics'][:80]}...\"")

    recommend_music("오늘 헤어져서 너무 슬픈데, 비까지 오네...")
    recommend_music("드디어 프로젝트가 끝났다! 밤새 드라이브하고 싶어!")

    print("\n\n4단계 완료: 챗봇 실행이 종료되었습니다.")
    print("="*50)


## 1단계: 데이터 준비 및 전처리 시작
총 51628개의 훈련 데이터와 6640개의 검증 데이터를 로드했습니다.
1단계 완료.

## 2단계: 감정 분류 모델 파인튜닝 시작


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/288 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/504 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

감정 분석 모델과 문맥 분석 모델을 로드했습니다.
URL에서 554개의 노래 데이터를 성공적으로 불러왔습니다.


Batches:   0%|          | 0/18 [00:00<?, ?it/s]


총 554곡의 가사로 듀얼 임베딩 데이터베이스를 구축했습니다.
3단계 완료.

## 4단계: 업그레이드된 감성 분석 챗봇 실행 시작 (Version: v1.5)

사용자 입력: '오늘 헤어져서 너무 슬픈데, 비까지 오네...'
-> 입력 감정 분석 결과: '슬픔'

--- 최종 추천 결과 ---
1. QWER - 눈물참기 (종합 점수: 0.6582)
   ㄴ 가사: "맑은 하늘에 비 내리는 날
내 마음과 정말 닮아서 이상하네요

한 방울 한 방울씩 떨어지는 비가
점점 맘에 차서 숨쉬기가 힘들 것 같아요

세상..."
2. 손담비 - 토요일밤에 (종합 점수: 0.6263)
   ㄴ 가사: "널 잃은 아픔에 찢어진 가슴에
텅 빈 내 마음에 난 이제 어떻게 살아
널 잃은 아픔에 찢어진 가슴에
텅 빈 내 마음에 한없이 슬퍼지는 오늘
토요..."
3. 제국의아이들 - 후유증 (종합 점수: 0.6151)
   ㄴ 가사: "이 지독한 후유증
먹지도 못하는
술을 자꾸 마시고
관심도 없는 애들한테
자꾸 연락하게 돼
외로워서 그런가봐
생각했는데
아무렇지 않게
지내다가도 ..."

사용자 입력: '드디어 프로젝트가 끝났다! 밤새 드라이브하고 싶어!'
-> 입력 감정 분석 결과: '기쁨'

--- 최종 추천 결과 ---
1. DAY6 - 한 페이지가 될 수 있게 (종합 점수: 0.5432)
   ㄴ 가사: "솔직히 말할게
많이 기다려 왔어
너도 그랬을 거라 믿어
오늘이 오길
매일같이 달력을 보면서
솔직히 나에게도
지금 이 순간은
꿈만 같아 너와 함께..."
2. IVE - I WANT (종합 점수: 0.5262)
   ㄴ 가사: "예이예이예 음
시끄럽게 쌓인 생각은 접어
두근대는 리듬 즐겨 like festa
가장 좋아하는 운동화를 신고
숨차게 달려봐
대체 누가 뭐가 그리..."
3. NMIXX - DICE (종합 점수: 0.5175)
   ㄴ 가사: "번쩍 뜬 눈을 깜빡
머린 핑핑핑 돌아
상상도 못한 drama
첫 scene을 찍어볼 time (go go)
이제 곧 시작될 ch

In [ ]:

# =================================================================================
# ## 4단계: 업그레이드된 감성 분석 챗봇 실행
# =================================================================================
if os.path.exists(f'/content/drive/MyDrive/chatbot_project/song_database_{VERSION}.json'):
    print("="*50)
    print(f"## 4단계: 업그레이드된 감성 분석 챗봇 실행 시작 (Version: {VERSION})")
    print("="*50)

    db_emotion_vectors = np.load(f'/content/drive/MyDrive/chatbot_project/lyrics_emotion_embeddings_{VERSION}.npy')
    db_context_vectors = np.load(f'/content/drive/MyDrive/chatbot_project/lyrics_context_embeddings_{VERSION}.npy')
    with open(f'/content/drive/MyDrive/chatbot_project/song_database_{VERSION}.json', 'r', encoding='utf-8') as f:
        db_song_info = json.load(f)

    def cosine_similarity(vec1, vec2_matrix):
        vec1_norm = np.linalg.norm(vec1)
        vec2_matrix_norm = np.linalg.norm(vec2_matrix, axis=1)
        epsilon = 1e-8
        return np.dot(vec2_matrix, vec1) / (vec1_norm * vec2_matrix_norm + epsilon)

    def recommend_music(user_input, top_k=3, emotion_weight=0.3, context_weight=0.7):
        print(f"\n사용자 입력: '{user_input}'")

        inputs = emotion_tokenizer(user_input, return_tensors="pt", truncation=True, padding=True, max_length=128)
        with torch.no_grad(): outputs = emotion_model(**inputs)

        user_emotion_label = emotion_model.config.id2label[torch.argmax(outputs.logits, dim=-1).item()]
        user_emotion_vector = outputs.hidden_states[-1][:, 0, :].numpy().flatten()
        user_context_vector = context_model.encode(user_input)

        print(f"-> 입력 감정 분석 결과: '{user_emotion_label}'")

        candidate_indices = [i for i, song in enumerate(db_song_info) if song['emotion'] == user_emotion_label]

        if not candidate_indices:
            print("-> 아쉽지만, 현재 감정과 일치하는 노래를 찾지 못했어요.")
            return

        cand_emotion_vectors = db_emotion_vectors[candidate_indices]
        cand_context_vectors = db_context_vectors[candidate_indices]
        candidate_songs = [db_song_info[i] for i in candidate_indices]

        emotion_sims = cosine_similarity(user_emotion_vector, cand_emotion_vectors)
        context_sims = cosine_similarity(user_context_vector, cand_context_vectors)

        total_scores = (emotion_weight * emotion_sims) + (context_weight * context_sims)
        sorted_indices = np.argsort(total_scores)[::-1]

        print("\n--- 최종 추천 결과 ---")
        for i in range(min(top_k, len(sorted_indices))):
            song_idx_in_cand = sorted_indices[i]
            song = candidate_songs[song_idx_in_cand]
            final_score = total_scores[song_idx_in_cand]
            print(f"{i+1}. {song['artist']} - {song['title']} (종합 점수: {final_score:.4f})")
            print(f"   ㄴ 가사: \"{song['lyrics'][:80]}...\"")

    recommend_music("오늘 헤어져서 너무 슬픈데, 비까지 오네...")
    recommend_music("드디어 프로젝트가 끝났다! 밤새 드라이브하고 싶어!")

    print("\n\n4단계 완료: 챗봇 실행이 종료되었습니다.")
    print("="*50)

## 4단계: 업그레이드된 감성 분석 챗봇 실행 시작 (Version: v1.3)

사용자 입력: '오늘 헤어져서 너무 슬픈데, 비까지 오네...'
-> 입력 감정 분석 결과: '슬픔'

--- 최종 추천 결과 ---
1. QWER - 눈물참기 (종합 점수: 0.6632)
   ㄴ 가사: "맑은 하늘에 비 내리는 날
내 마음과 정말 닮아서 이상하네요

한 방울 한 방울씩 떨어지는 비가
점점 맘에 차서 숨쉬기가 힘들 것 같아요

세상..."
2. 손담비 - 토요일밤에 (종합 점수: 0.6310)
   ㄴ 가사: "널 잃은 아픔에 찢어진 가슴에
텅 빈 내 마음에 난 이제 어떻게 살아
널 잃은 아픔에 찢어진 가슴에
텅 빈 내 마음에 한없이 슬퍼지는 오늘
토요..."
3. 제국의아이들 - 후유증 (종합 점수: 0.6237)
   ㄴ 가사: "이 지독한 후유증
먹지도 못하는
술을 자꾸 마시고
관심도 없는 애들한테
자꾸 연락하게 돼
외로워서 그런가봐
생각했는데
아무렇지 않게
지내다가도 ..."

사용자 입력: '드디어 프로젝트가 끝났다! 밤새 드라이브하고 싶어!'
-> 입력 감정 분석 결과: '기쁨'

--- 최종 추천 결과 ---
1. DAY6 - 한 페이지가 될 수 있게 (종합 점수: 0.5469)
   ㄴ 가사: "솔직히 말할게
많이 기다려 왔어
너도 그랬을 거라 믿어
오늘이 오길
매일같이 달력을 보면서
솔직히 나에게도
지금 이 순간은
꿈만 같아 너와 함께..."
2. IVE - I WANT (종합 점수: 0.5432)
   ㄴ 가사: "예이예이예 음
시끄럽게 쌓인 생각은 접어
두근대는 리듬 즐겨 like festa
가장 좋아하는 운동화를 신고
숨차게 달려봐
대체 누가 뭐가 그리..."
3. 여자친구 - 너 그리고 나 (종합 점수: 0.5214)
   ㄴ 가사: "알 수 있었어 널 본 순간
뭔가 특별하다는 걸
눈빛 만으로도 느껴지니까
마음이 움직이는 걸
나비처럼 날아 나나나 나빌레라
바람아 바람아 불어라
..."


4단계 완료: 챗봇 실행이 종료되었습니다.
